In [1]:
%store -r normalized_df

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
X = normalized_df.drop(['kreftform'], axis=1)
y = normalized_df[['kreftform']]

In [128]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn import utils

data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

lab_enc = preprocessing.LabelEncoder()
encoded_y_train = lab_enc.fit_transform(y_train.values.ravel())
encoded_y_test = lab_enc.fit_transform(y_test.values.ravel())

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train,encoded_y_train)

y_pred = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(encoded_y_test, y_pred))
print("RMSE: %f" % (rmse))

predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(encoded_y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

RMSE: 0.482554
Accuracy: 79.56%
